In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
#포항시 데이터 불러오기 및 기본 정제 시작

In [49]:
migration_df  = pd.read_csv('/content/drive/MyDrive/데이터마이닝/기말/github/Data_Analysis_Project/포항시데이터/시군구별_이동자수_20250617165942.csv')
industry_df  = pd.read_csv('/content/drive/MyDrive/데이터마이닝/기말/github/Data_Analysis_Project/포항시데이터/시군구_산업별_취업자_10차__20250617165353.csv')
age_df   = pd.read_csv('/content/drive/MyDrive/데이터마이닝/기말/github/Data_Analysis_Project/포항시데이터/시군구_연령별_취업자_및_고용률__전체__20250617165132.csv')

In [51]:
migration_df.head()

,행정구역(시군구)별,2015.01,2015.01.1,2015.01.2,2015.01.3,2015.01.4,2015.01.5,2015.01.6,2015.01.7,2015.02,...,2024.3/4.6,2024.3/4.7,2024.4/4,2024.4/4.1,2024.4/4.2,2024.4/4.3,2024.4/4.4,2024.4/4.5,2024.4/4.6,2024.4/4.7
0,행정구역(시군구)별,총전입 (명),총전출 (명),순이동 (명),시도내이동-시군구내 (명),시도내이동-시군구간 전입 (명),시도내이동-시군구간 전출 (명),시도간전입 (명),시도간전출 (명),총전입 (명),...,시도간전입 (명),시도간전출 (명),총전입 (명),총전출 (명),순이동 (명),시도내이동-시군구내 (명),시도내이동-시군구간 전입 (명),시도내이동-시군구간 전출 (명),시도간전입 (명),시도간전출 (명)
1,포항시,4901,5692,-791,2722,611,729,1568,2241,5593,...,3732,3950,14221,14167,54,9225,1301,1146,3695,3796


In [52]:
migration_df.columns

Index(['행정구역(시군구)별', '2015.01', '2015.01.1', '2015.01.2', '2015.01.3',
       '2015.01.4', '2015.01.5', '2015.01.6', '2015.01.7', '2015.02',
       ...
       '2024.3/4.6', '2024.3/4.7', '2024.4/4', '2024.4/4.1', '2024.4/4.2',
       '2024.4/4.3', '2024.4/4.4', '2024.4/4.5', '2024.4/4.6', '2024.4/4.7'],
      dtype='object', length=1313)

In [54]:
industry_df

,행정구역별,산업별,2015.1/2,2015.2/2,2016.1/2,2016.2/2,2017.1/2,2017.2/2,2018.1/2,2018.1/2.1,...,2021.2/2.1,2022.1/2,2022.2/2,2022.2/2.1,2023.1/2,2023.1/2.1,2023.2/2,2023.2/2.1,2024.1/2,2024.1/2.1
0,행정구역별,산업별,취업자 (천명),취업자 (천명),취업자 (천명),취업자 (천명),취업자 (천명),취업자 (천명),취업자 (천명),RSE,...,RSE,취업자 (천명),취업자 (천명),RSE,취업자 (천명),RSE,취업자 (천명),RSE,취업자 (천명),RSE
1,포항시,계,255,257,257,258,262,255,260,-,...,-,259,255,-,253,-,253,-,257,-
2,포항시,"농업, 임업 및 어업 (A)",22,19,19,19,21,17,16,*,...,*,21,15,*,14,*,10,*,9,*
3,포항시,"광·제조업(B,C)",54,51,53,56,52,51,53,-,...,-,53,53,-,53,-,52,-,52,-
4,포항시,건설업 (F),18,20,21,21,22,19,19,-,...,-,17,23,-,23,-,24,-,22,-
5,포항시,"도소매·숙박음식업(G,I)",52,52,53,53,54,56,58,-,...,-,44,44,-,44,-,46,-,49,-
6,포항시,"전기·운수·통신·금융(D,H,J,K)",27,24,24,25,25,22,23,-,...,-,22,21,-,21,-,21,-,23,-
7,포항시,"사업·개인·공공서비스 및 기타(E,L~U)",83,90,87,85,88,90,91,-,...,-,102,100,-,99,-,101,-,102,-


In [65]:

import pandas as pd
import re

# 1) CSV 읽기 ─ 첫 두 줄을 다중 헤더로 해석
path = "/content/drive/MyDrive/데이터마이닝/기말/github/Data_Analysis_Project/포항시데이터/시군구별_이동자수_20250617165942.csv"
raw = pd.read_csv(path, header=[0, 1])

# 2) ‘YYYY.MM’ 형식 & 관심 지표(총전입·총전출·순이동)만 선택
metrics = ['총전입 (명)', '총전출 (명)', '순이동 (명)']
month_pat = re.compile(r'\d{4}\.\d{2}$')

month_cols = [(d, g) for d, g in raw.columns
              if month_pat.match(d) and g in metrics]
df_month = raw.loc[:, month_cols]

# 3) 월표시 → 반기 라벨 변환
def month_to_half(label: str) -> str:
    y, m = label.split('.')
    return f"{y}.{1 if int(m) <= 6 else 2}/2"

df_month.columns = pd.MultiIndex.from_tuples(
    [(month_to_half(d), g) for d, g in df_month.columns]
)

# 4) (반기·지표)별로 **중복 컬럼을 먼저 합산**해 고유화[4]
df_month = df_month.groupby(level=[0, 1], axis=1).sum()

# 5) 모든 시·군·구 행을 더해 전국 합계 → (반기, 지표) MultiIndex Series
half_totals = df_month.sum(axis=0)

# 6) 반기를 인덱스, 지표를 열로 재구성[8]
df_half = half_totals.unstack(level=1).sort_index()

print(df_half)     # 원하는 형태


          순이동 (명)  총전입 (명)  총전출 (명)
2015.1/2    -2414    30859    33273
2015.2/2      789    33290    32501
2016.1/2    -2329    28642    30971
2016.2/2    -1857    24524    26381
2017.1/2    -2298    27008    29306
2017.2/2    -1382    23949    25331
2018.1/2    -2608    30061    32669
2018.2/2    -1364    26752    28116
2019.1/2    -1427    29985    31412
2019.2/2    -1317    24691    26008
2020.1/2    -2367    34469    36836
2020.2/2    -1160    26532    27692
2021.1/2     1797    32585    30788
2021.2/2      791    28658    27867
2022.1/2    -3214    25418    28632
2022.2/2    -2157    21290    23447
2023.1/2    -2299    24290    26589
2023.2/2      171    23085    22914
2024.1/2      220    32319    32099
2024.2/2      -35    26640    26675
2025.1/2     -734    20087    20821


<ipython-input-65-358469153>:26: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_month = df_month.groupby(level=[0, 1], axis=1).sum()


In [66]:
df_half.to_csv('/content/drive/MyDrive/데이터마이닝/기말/github/Data_Analysis_Project/포항시데이터/preprocessed_migration_Pohang_data.csv')